# Attach a LoRA adapter and train

## Setup & Imports

In [1]:
import os
from uuid import uuid4

import torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

from twitter_emoji_reaction_lora.data import load_emoji_dataset, tokenize_and_format
from twitter_emoji_reaction_lora.model import build_base_model, build_peft_model
from twitter_emoji_reaction_lora.utils import print_trainable_parameters, compute_metrics

/home/jsooh/ssd/t5/envs/conda_envs/twitter-emoji/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load & Tokenize  
We pull in the same `load_emoji_dataset` and `tokenize_and_format` functions we used in the 01_explore_tokenize notebook.


In [2]:
# load raw splits
ds = load_emoji_dataset()

# apply our shared tokenization routine
ds_tok, tok = tokenize_and_format(ds, max_length=128)


## 3. Model Initialization & LoRA  
Here we use `build_base_model` to get a `RobertaForSequenceClassification`, then wrap it in LoRA via `build_peft_model`.


In [3]:
model = build_base_model()
print_trainable_parameters(model)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 124661012 || all params: 124661012 || trainable%: 100.00


In [4]:
lora_model = build_peft_model(model)
print_trainable_parameters(lora_model)

trainable params: 2375444 || all params: 127036456 || trainable%: 1.87


## 4. Trainer Setup  
Then we prepare the model for training!


In [6]:
model_id = f"roberta-base-with-tweet-eval-emoji"

training_args = TrainingArguments(
    output_dir=model_id,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    warmup_ratio=0.05,
    num_train_epochs=4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=256,
    max_grad_norm=0.5,
    label_smoothing_factor=0.1,
    save_total_limit=3,
    logging_steps=30,
    fp16=True,
    push_to_hub=False,
    report_to="wandb",
    run_name = f"emoji-{uuid4().hex[:8]}",
    label_names=["labels"],
)
data_collator = DataCollatorWithPadding(tok, pad_to_multiple_of=8)

In [135]:
from transformers import Trainer
from sklearn.utils.class_weight import compute_class_weight

labels = np.array(ds_tok["train"]["labels"])
weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(weights, dtype=torch.float)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = torch.nn.functional.cross_entropy(
            logits, labels, weight=class_weights.to(logits.device)
        )
        return (loss, outputs) if return_outputs else loss


In [136]:
import wandb

trainer = WeightedTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Top3 Accuracy
1,2.140100,2.417112,0.222400,0.220445,0.415200
2,2.115700,2.372503,0.241200,0.246300,0.437600
3,2.014300,2.351388,0.250800,0.255062,0.444800
4,1.934600,2.357327,0.259400,0.263203,0.450000


eval/accuracy,▁▅▆█
eval/f1,▁▅▇█
eval/loss,█▃▁▂
eval/runtime,▃█▅▁
eval/samples_per_second,▆▁▄█
eval/steps_per_second,▆▁▄█
eval/top3_accuracy,▁▆▇█
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▁▂▅▄▆▆▄▃▆▄▄▅▄▅▆█▆▅▅▆▄▆▆▅▇▆▆▆▇▅▇▆▆▇██▆█▆█
train/learning_rate,▄▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁


In [139]:
metrics = trainer.evaluate(ds_tok["test"])
print(metrics)

# save weights and tokenizer
lora_model.save_pretrained(model_id)
tok.save_pretrained(model_id)

{'eval_loss': 1.9591624736785889, 'eval_accuracy': 0.4286, 'eval_f1': 0.33462876199193464, 'eval_top3_accuracy': 0.65028, 'eval_runtime': 104.9624, 'eval_samples_per_second': 476.361, 'eval_steps_per_second': 1.867, 'epoch': 4.0}


('roberta-base-with-tweet-eval-emoji-full/tokenizer_config.json',
 'roberta-base-with-tweet-eval-emoji-full/special_tokens_map.json',
 'roberta-base-with-tweet-eval-emoji-full/vocab.json',
 'roberta-base-with-tweet-eval-emoji-full/merges.txt',
 'roberta-base-with-tweet-eval-emoji-full/added_tokens.json',
 'roberta-base-with-tweet-eval-emoji-full/tokenizer.json')

In [ ]:
wandb.finish()

In [140]:
!ls -lh {model_id}

total 14M
-rw-r--r-- 1 root root  870 Jun 23 18:44 adapter_config.json
-rw-r--r-- 1 root root 9.1M Jun 23 18:44 adapter_model.safetensors
drwxr-xr-x 2 root root 4.0K Jun 23 18:32 checkpoint-1056
drwxr-xr-x 2 root root 4.0K Jun 23 16:59 checkpoint-1408
drwxr-xr-x 2 root root 4.0K Jun 23 17:03 checkpoint-2112
-rw-r--r-- 1 root root 446K Jun 23 18:44 merges.txt
-rw-r--r-- 1 root root 5.0K Jun 23 18:44 README.md
-rw-r--r-- 1 root root  280 Jun 23 18:44 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K Jun 23 18:44 tokenizer_config.json
-rw-r--r-- 1 root root 3.4M Jun 23 18:44 tokenizer.json
-rw-r--r-- 1 root root 780K Jun 23 18:44 vocab.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [141]:
from peft import PeftConfig, PeftModel

config = PeftConfig.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=20
)
inference_model = PeftModel.from_pretrained(model, model_id).eval()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [142]:
id2label = {
    0: "❤", 1: "😍", 2: "😂", 3: "💕", 4: "🔥",
    5: "😊", 6: "😎", 7: "✨", 8: "💙", 9: "😘",
    10: "📷", 11: "🇺🇸", 12: "☀", 13: "💜", 14: "😉",
    15: "💯", 16: "😁", 17: "🎄", 18: "📸", 19: "😜"
}

In [152]:
from transformers import pipeline

pipe = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tok,
    return_all_scores=True,          # return all 20 logits
    function_to_apply="softmax"      # turn logits -> probabilities
)

def emojify(text, k=3):
    probs = pipe(text)[0]
    top = sorted(probs, key=lambda x: x["score"], reverse=True)[:k]
    return " ".join(id2label[int(d["label"].split("_")[-1])] for d in top)


print(emojify("Sunny day"))

Device set to use cuda:0


☀ 💙 😎


In [153]:
repo_id = "codinglabsong/roberta-base-tweet-emoji-lora"

trainer.create_model_card(   # adds automatic metrics, dataset info, tags
    model_name=repo_id.split("/")[-1],
    language="en",
    license="apache-2.0",
    tags=["twitter", "emoji", "lora", "roberta-base"],
)

trainer.push_to_hub(
    repo_id,                        # full path, e.g. "your-hf-username/…"
    finetuned_from="roberta-base",
    tasks="text-classification",
    dataset="tweet_eval/emoji",
)

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/9.51M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/codinglabsong/roberta-base-with-tweet-eval-emoji-full/commit/76696d41cb9ae04b75b3324ac2c921f5b055ae1c', commit_message='codinglabsong/roberta-base-tweet-emoji-lora', commit_description='', oid='76696d41cb9ae04b75b3324ac2c921f5b055ae1c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/codinglabsong/roberta-base-with-tweet-eval-emoji-full', endpoint='https://huggingface.co', repo_type='model', repo_id='codinglabsong/roberta-base-with-tweet-eval-emoji-full'), pr_revision=None, pr_num=None)

In [159]:

adapter_id = "codinglabsong/roberta-base-tweet-emoji-lora"
base_id    = "roberta-base"

# 1) tokenizer
tok = AutoTokenizer.from_pretrained(adapter_id)

# 2) base backbone (suppress that first warning)
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_id,
    num_labels=20,
    ignore_mismatched_sizes=True,   # hides “not initialized” notice
)

# 3) load LoRA
model = PeftModel.from_pretrained(base_model, adapter_id)

# OPTIONAL: merge so the pipeline sees a plain Roberta model
model = model.merge_and_unload()    # now type(model) == RobertaForSequenceClassification
model.eval()

# 4) build pipeline (no “unsupported” warning anymore)
pipe = pipeline(
    "text-classification",
    model=model,
    tokenizer=tok,
    return_all_scores=True,
    function_to_apply="softmax",
)

def emojify(text, k=3):
    scores = pipe(text)[0]
    top = sorted(scores, key=lambda x: x["score"], reverse=True)[:k]
    return " ".join(id2label[int(s["label"].split("_")[-1])] for s in top)

print(emojify("Sunny day"))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


☀ 💙 😎


| Area                              | What’s happening in your notebook                 | Why it hurts                                                                                            | How to fix                                                                                                                   |
| --------------------------------- | ------------------------------------------------- | ------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------- |
| **Evaluation flag typo**          | `eval_strategy="epoch"`                           | HF ignores this; no mid-epoch evaluation or early stopping signals.                                     | Rename to `evaluation_strategy="epoch"` (or `"steps"`).                                                                      |
| **Learning-rate & batch size**    | `lr=9 e-4`, `batch=256`                           | Very high LR + very large batch ⇒ under-fitting & unstable loss (your val loss 2.57 vs baseline ≈ 1.6). | Try **2 e-4 – 3 e-4** with batch 32 (or keep 256 but add `gradient_accumulation_steps` so the *effective* batch is smaller). |
| **LoRA only, small base model**   | LoRA on DeBERTa-v3-small                          | You froze most weights; for 20-way classification, the extra capacity matters.                          | **Full fine-tune** the base model first (or use DeBERTa-v3-base / RoBERTa-base). Once that is solid, experiment with LoRA.   |
| **LoRA target list**              | `["query_proj","value_proj", … ]` (no `key_proj`) | Missing keys ⇒ only part of self-attention adapts.                                                      | Add `"key_proj"` (and verify exact layer names with `print(model)`); or let 🤗 PEFT auto-match by regex.                     |
| **Max sequence length**           | `max_length=70`                                   | \~25 % of tweets in this set are longer; you’re truncating signal.                                      | Increase to **128** (Twitter limit is 280 chars; 128 covers > 97 % after tokenisation).                                      |
| **Class imbalance**               | Plain CE loss                                     | Macro-F1 punishes you for rare emojis (some appear < 1 % in train).                                     | Pass **class weights** to loss (or use **focal loss**). HF `Trainer` lets you supply a custom `loss_fn`.                     |
| **Weight decay & regularisation** | Not set                                           | Over-fitting the dominant classes, poor generalisation.                                                 | Add `weight_decay=0.01` and/or `label_smoothing_factor=0.1`.                                                                 |
| **Dynamic padding**               | Padding to 70 tokens everywhere wastes compute.   | Fewer real tokens per batch ⇒ lower throughput.                                                         | Use `DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)` for mixed-precision efficiency.                               |
